In [6]:
# imports and uploads
# !pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

from google.colab import files
files.upload()

with open('labelled_bedrooms_data_sample.txt.txt') as f:
    bedrooms_lines = f.readlines()

bedroom_files = []
for bedroom_line in bedrooms_lines:
    bedroom_files.append(bedroom_line.split("; ")[0])
bedrooms = []
for bedroom_line in bedrooms_lines:
    bedrooms.append(bedroom_line.split("; ")[2:])

def get_vector_representation(vector, nlp):
    vec = []
    for phrase in vector:
        tokens = nlp(phrase)
        for token in tokens:
            word = token.text
            if word in nlp.vocab:
                vec.extend(nlp(word).vector)
    return vec

!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load("en_core_web_lg")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving labelled_bedrooms_data_sample.txt.txt to labelled_bedrooms_data_sample.txt (3).txt
2023-11-19 22:19:28.969013: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 22:19:28.969097: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 22:19:28.969133: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 22:19:28.977687: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFl

In [9]:
# creating static inputs to test our program
user_string = 'Little bedroom with yellow double ottoman and light table and picture'.split()
user_input = 'Little bedroom with yellow double ottoman and light table and picture'
user_vec = get_vector_representation(user_string, nlp)

The following code looks for the synonims of inputed by a user words, if they are met more frequently in the labelled data (actually data was labbeled in a way to avoid synonyms, using only one term for same or similar items). If there is a synonym for the word, that is used in labelled data, but not in user's quiry, the code replaces the word in the user query with a synonym, found in labelled data.

In [10]:
labeled_data_beds = [word for ngram in bedrooms for phrase in ngram for word in phrase.split()]

def replace_synonyms(user_input, labeled_words):
    tokens = word_tokenize(user_input)
    replaced_tokens = []
    for token in tokens:
        if token not in labeled_words:
            synonyms = set()
            for syn in wordnet.synsets(token):
                for lemma in syn.lemmas():
                    synonym = lemma.name().lower()
                    if synonym in labeled_words:
                        synonyms.add(synonym)
            if synonyms:
                replaced_tokens.append(synonyms.pop())
            else:
                replaced_tokens.append(token)
        else:
            replaced_tokens.append(token)

    replaced_text = ' '.join(replaced_tokens)
    return replaced_text

result = replace_synonyms(user_input, labeled_data_beds)
print(result)

small bedroom with yellow double pouf and light table and picture and candelabrum
